In [ ]:
try:
    # settings colab:
    import google.colab
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# LAB: Pipelines y Transformadores


## 1. Introducción
En este Lab vamos a crear Pipelines para pre-procesar datos y extraer características sobre el [Titanic dataset](http://www.kaggle.com/c/titanic-gettingStarted/data).

El dataset es una lista de pasajeros del trasatlántico más famoso. La segunda columna del dataset ("survived") indica si la persona ha sobrevivido (1) o no (0) al naufragio. El resto de las columnas contienen información diversa sobre cada uno de los pasajeros.

* Levantamos el dataset (Titanic.csv) en un DataFrame.

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

df = pd.read_csv('../Data/Titanic.csv')
df.head()

Usemos el método info para dar un vistazo a los datos:

- Hay características numéricas?
- Hay características categóricas?
- Hay datos incompletos? En qué columnas?
- Cuáles te parecen importantes para rellenar?

In [ ]:
df.info()

In [ ]:
df.Survived.value_counts()

In [ ]:
df.isnull().sum()

# 2. Preprocesando cada grupo de columnas

Observamos que el preprocesamiento de los datos requiere distintos enfoques para distintos tipos de columnas: algunas requieren imputación, otras requieren generar variables Dummies y otras sería conveniente estandarizarlas.

La idea es armar un pipeline separado para el preprocesamiento que necesita cada grupo de variables y luego unirlos todos con el método [`make_union`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_union.html) que ejecutará todos los pipelines para luego concatenar el resultado.

Para hacer las transformaciones de cada grupo de columnas sugerimos crear un transformer de sklearn ColumnSelector que permita seleccionar un grupo de columnas del DataFrame donde queremos aplicar las transformaciones.



## 2. Edad

Se puede observar que hay varios pasajeros sin información de edad (columna "Age"). Vamos a intentar llenar los datos de esta columna. Exploremos la distribución de valores para los datos existentes y pensemos una estrategia.

In [ ]:
df.Age.plot(kind = 'hist');

### 2.b Transformador de Edad

Dependiendo la estrategia que hayamos decido vamos a necesitar imputar los datos de edad faltantes, ya sea usando un transformador del módulo de pre-procesamiento o crear un transformador custom transformer.
Esto podría implicar:

- Seleccionar una o más columnas
- Llenar los datos faltantes
- Escalar los valores de Edad

In [ ]:
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
    
    def transform(self, X, *_):
        if isinstance(X, pd.DataFrame):
            return pd.DataFrame(X[self.columns])
        else:
            raise TypeError("Este Transformador solo funciona en DF de Pandas")
    
    def fit(self, X, *_):
        return self
    
cs = ColumnSelector('Age')

cs.transform(df).head()

In [ ]:
age_pipe = make_pipeline(ColumnSelector('Age'),
                         SimpleImputer(),
                         StandardScaler())

age_pipe.fit_transform(df)

## 3. Variables Categóricas

"Embarked" y "Pclass" son variables categóricas. Usá la función [`get_dummies`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) de pandas para crear columnas correspondientes a los valores de las mismas.

"Embarked" tiene un par de datos faltantes. Llenalos con el puerto de embarque más común en el dataset.

Sugerencia: Crear un transformador custom que "envuelva" el uso de get_dummies.

In [ ]:
df.Embarked.value_counts()

In [ ]:
df.Embarked = df.Embarked.fillna('S')

In [ ]:
df['Embarked'].value_counts()

In [ ]:
pd.get_dummies(pd.DataFrame(df['Embarked'].head()))

In [ ]:
class GetDummiesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
    
    def transform(self, X, *_):
        if isinstance(X, pd.DataFrame):
            return pd.get_dummies(X[self.columns], columns = self.columns, drop_first = True)
        else:
            raise TypeError("Este Transformador solo funciona en DF de Pandas")
    
    def fit(self, X, *_):
        return self
    
gdt = GetDummiesTransformer(['Embarked'])
gdt.fit_transform(df).head()

In [ ]:
dummies_pipe = make_pipeline(GetDummiesTransformer(['Pclass', 'Embarked', 'Sex']),
                             StandardScaler())

dummies_pipe.fit_transform(df)[:5,:]

## 4. Tarifa

Escalar el atributo "Fare" (Tarifa) usando uno de los escaladores existentes en el módulo de preprocesamiento. 

In [ ]:
fare_pipe = make_pipeline(ColumnSelector('Fare'),
                          StandardScaler())

fare_pipe.fit_transform(df.head())

## 5. Otras columnas

In [ ]:
other_pipe = make_pipeline(ColumnSelector(['SibSp', 'Parch']))

## 6. Union

Utilizá una FeatureUnion o la función make_union para combinar todos los pipelines que has creado.

In [ ]:
union = make_union(age_pipe,
                   dummies_pipe,
                   fare_pipe,
                   other_pipe)

union.fit_transform(df.head())

## 7. Predicciones

Ahora utilicemos GridSearch para evaluar la performance de estas transformaciones, seguidas de un modelo de regresión logística. 
Para esto exploren distintos valores de parámetros para C.


Pimero separar las X y las y

In [ ]:
X = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y = df['Survived']
X.head()

Separar los datos de train y test

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

Aplicar el `pipeline` con todos los objetos transformadores del dataframe

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [ ]:
pipeline = Pipeline([('union', union), 
                     ('log', LogisticRegression(solver='liblinear'))])

parameters = {'log__penalty': ['l1', 'l2'],
              'log__C': [0.01, 0.1, 1, 10, 100]}

In [ ]:
from sklearn.model_selection import StratifiedKFold

folds=StratifiedKFold(n_splits=5,shuffle=True, random_state=42)

grid_search = GridSearchCV (pipeline, parameters,cv=folds, n_jobs = 5)

In [ ]:
grid_search.fit(pd.DataFrame(X_train, columns=X.columns), y_train)

In [ ]:
print("Best score: %0.3f" % grid_search.best_score_) 
print("Best parameters set:" )
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted (parameters . keys()): 
                    print("\t %s: %r" % (param_name, best_parameters[param_name])) 

#### Performance sobre datos nuevos



In [ ]:
y_pred = grid_search.best_estimator_.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))